<a href="https://colab.research.google.com/github/giggling-shivam/RAG_For_Every_Language/blob/main/RAG_google_genAi_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install unstructured
!pip install google-generativeai
!pip install chromadb
!pip install tiktoken
!pip install --upgrade --quiet  langchain-google-genai
!pip install faiss-gpu
!pip install text_generation
!pip install langchain_huggingface
!pip install huggingface_hub
!pip install accelerate
!pip install transformers
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.

In [ ]:
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def directory_data(data_path):
  DATA_PATH = data_path
  doc = DirectoryLoader(DATA_PATH, glob="*.md").load()
  textsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500, add_start_index=True)
  docs = textsplitter.split_documents(doc)
  return docs

def online_data(url):
  pdf_file_url = url
  loader = PyPDFLoader(pdf_file_url)
  textsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500, add_start_index=True)
  docs = loader.load_and_split(textsplitter)
  return docs

data_source = 'online'

if data_source == 'online':                                   #if the document is scrapped through an URL
  docs = online_data('https://arxiv.org/pdf/2402.04049')
else:                                                         #if the document is saved in an existing Directory
  docs = directory_data('/content/sample_data')

print('document content would look like- ', docs[0].page_content)

document content would look like-  Systematic Biases in LLM Simulations of Debates
Amir Taubenfeld12∗ Yaniv Dover34 Roi Reichart5 Ariel Goldstein236
*Corresponding Author: amirt@google.com
1The Hebrew University of Jerusalem, School of Computer Science and Engineering
2Google Research
3The Hebrew University Business School, Jerusalem, Israel
4Federmann Center for the Study of Rationality, Hebrew University, Jerusalem, Israel
5Faculty of Data and Decision Sciences, Technion
6Department of Cognitive and Brain Sciences, Hebrew University, Jerusalem, Israel
Abstract
The emergence of Large Language Models
(LLMs), has opened exciting possibilities for
constructing computational simulations de-
signed to replicate human behavior accurately.
Current research suggests that LLM-based
agents become increasingly human-like in their
performance, sparking interest in using these
AI agents as substitutes for human participants
in behavioral studies. However, LLMs are com-
plex statistical learners wi

In [ ]:
import os
import shutil
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
goggle_embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')


def use_chromaDB(docs, query_text, embedding): #biggner friendly, vector embeddings & metadata based search, searching technique - cosine similarity
  # DATABASE= "/content/sample_data/chroma"
  # if os.path.exists(DATABASE):
  #   shutil.rmtree(DATABASE)
  # db = Chroma.from_documents(docs, embedding, persist_directory=DATABASE)
  db = Chroma.from_documents(docs, embedding)
  relevant_context = db.similarity_search_with_relevance_scores(query_text, k=1)
  page_contents = [doc.page_content for doc, _ in relevant_context]
  context_text = '\n\n---\n\n'.join(page_contents)
  return context_text

def use_FAISS(docs,query_text, embedding): #high performance, highly scalable, low latency,pure vector embeddings no metadata based search, seqarching technique - K nearest Neighbour
  db = FAISS.from_documents(docs, embedding)
  relevant_context = db.similarity_search_with_relevance_scores(query_text, k=1)
  page_contents = [doc.page_content for doc, _ in relevant_context]
  context_text = '\n\n---\n\n'.join(page_contents)
  return context_text

query_text = 'what is llm'
context_text = use_chromaDB(docs, query_text, goggle_embeddings)
print(context_text)

serve as invaluable tools for exploring the intrica-
cies of human interactions and decision-making
processes. This would allow scientists to conduct
their research with speed and efficiency, substan-
tially lowering the considerable resources usually
needed for recruiting and analyzing human subjects.
Consequently, a range of studies have demonstrated
the promise of these simulations across various
disciplines, including human psychology (Dillion
et al., 2023), social dynamics (Park et al., 2022),
and economics (Horton, 2023; Chen et al., 2023).
However, LLMs are complex statistical learn-
ers that do not depend on straightforward deduc-
tive rules. Despite exhibiting impressive emerging
skills that challenge our current understanding of
cognition (Wei et al., 2022; Bubeck et al., 2023),
their indeterminate nature leaves them susceptible
to unintended behaviors. One example is their man-
ifestation of inherent biases, including gender bias


In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain import LLMChain, PromptTemplate
import google.generativeai as genai
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import OpenLLM
from langchain_huggingface import HuggingFaceEndpoint
import warnings
warnings.filterwarnings("ignore")


query_text= '「大規模言語モデルとは何ですか？」' # what is llm

def gemini_llm(context_text, query_text):
  os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
  genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

  template = """Answer the question only in the context below. if the context is
  not provided in english language then detect the language and convert it to english language.

  this is the context:
  {context}

  ---

  Answer the question only on the above context and in language detected, if the question is not provided in english language then convert it to english language.
  this is the question: {question}"""

  prompt = ChatPromptTemplate.from_template(template).format(context=context_text, question=query_text)
  model = genai.GenerativeModel('gemini-1.5-flash')
  result = model.generate_content(prompt)
  candidate = result.candidates
  generated_text = candidate[0].content.parts[0].text
  print(generated_text)


def huggingFace_llm(context_text, query_text):
  os.environ['HUGGINGFACEHUB'] = userdata.get('HUGGINGFACEHUB')
  template = """Answer the question only in the context below. if the context is
  not provided in english language then detect the language and convert it to english language.

  this is the context:
  {context}

  ---

  Answer the question only on the above context, if the question is not in english language then convert it to english language.
  this is the question: {question}"""

  prompt = PromptTemplate(template=template, input_variables=[context_text, query_text])

  repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
  llm = HuggingFaceEndpoint(repo_id=repo_id,temperature=0.7, token=os.environ['HUGGINGFACEHUB'])
  chain = prompt | llm
  result = chain.invoke({'context':context_text, 'question':query_text})
  return result.split('Answer')[1].strip()



In [ ]:
# print(gemini_llm(context_text, query_text))
print(huggingFace_llm(context_text, query_text))


                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


:
  A large-scale language model (LLM) is a type of artificial intelligence system designed to understand and generate human language. It learns from vast amounts of text data to identify patterns and make predictions or completions based on given input. In the provided context, it is highlighted as an essential tool for scientific research, particularly in the fields of psychology, social dynamics, and economics, due to its ability to efficiently explore human interactions and decision-making processes, and its demonstrated promise across various disciplines. However, LLMs are complex statistical learners that can exhibit unintended behaviors, such as inherent biases.
